In [3]:
import requests
from dotenv import load_dotenv
import os

# Load environment variables from .env file
load_dotenv()
APIFY_TOKEN = os.getenv("APIFY_API_KEY")

# Ensure the token is present
if not APIFY_TOKEN:
    raise OSError("APIFY_API_KEY not found in environment variables.")

# TikTok video URL to scrape
tiktok_url = "https://www.tiktok.com/@cioccatalin1/video/7456347160892706070"

# API endpoint
run_url = f"https://api.apify.com/v2/acts/clockworks~tiktok-comments-scraper/run-sync-get-dataset-items?token={APIFY_TOKEN}"

# Input payload for the actor
payload = {
    "postURLs": [tiktok_url],
    "commentsPerPost": 50,
    "maxRepliesPerComment": 0,
    "resultsPerPage": 50,
}

# Make the POST request
response = requests.post(run_url, json=payload)
response.raise_for_status()
comments = response.json()

# Print a few comments
for i, comment in enumerate(comments[:5], start=1):
    print(f"{i}. {comment.get('text')} (by {comment.get('username')})")

1. La Ferrari più bella che ci sia!! (by None)
2. ma quanto è bella sempre intramontabile 💪 (by None)
3. ma davvero c'e' chi preferisce la countach? (by None)
4. Le supersportive Italiane,sono le migliori in assoluto....... (by None)
5. Bellezza e fascino senza tempo😍😍😍, la Ferrari testa rossa ha linee belle. che sembra uscita ora dalla fabbrica (by None)


In [5]:
"""
Fetch TikTok search results + comments in one go.

Requires:
    pip install requests python-dotenv
"""

from __future__ import annotations
import os
import requests
from dotenv import load_dotenv
from typing import List, Dict, Any

# ──────────────────────────────────────────────────────────────────────────────
#  Environment & constants
# ──────────────────────────────────────────────────────────────────────────────
load_dotenv()
APIFY_TOKEN = os.getenv("APIFY_API_KEY")
if not APIFY_TOKEN:
    raise OSError("APIFY_API_KEY not found in environment variables (.env)")

SEARCH_ACTOR = "epctex~tiktok-search-scraper"
COMMENT_ACTOR = "clockworks~tiktok-comments-scraper"

SEARCH_URL = (
    f"https://api.apify.com/v2/acts/{SEARCH_ACTOR}/"
    f"run-sync-get-dataset-items?token={APIFY_TOKEN}"
)
COMMENT_URL = (
    f"https://api.apify.com/v2/acts/{COMMENT_ACTOR}/"
    f"run-sync-get-dataset-items?token={APIFY_TOKEN}"
)


# ──────────────────────────────────────────────────────────────────────────────
#  Helpers
# ──────────────────────────────────────────────────────────────────────────────
def _search_tiktok(keyword: str, max_items: int = 10) -> list[dict[str, Any]]:
    """Run the search actor and return raw dataset items."""
    payload = {
        "search": [keyword],
        "maxItems": max_items,
        # The actor’s input schema requires `proxy`
        "proxy": {"useApifyProxy": True},
    }
    resp = requests.post(SEARCH_URL, json=payload, timeout=300)
    resp.raise_for_status()
    return resp.json()  # list of video objects


def _fetch_comments(video_url: str, limit: int = 20) -> list[dict[str, Any]]:
    """Run the comment actor for a single TikTok video URL."""
    payload = {
        "postURLs": [video_url],
        "commentsPerPost": limit,
        "maxRepliesPerComment": 0,
        "resultsPerPage": limit,
    }
    resp = requests.post(COMMENT_URL, json=payload, timeout=300)
    resp.raise_for_status()
    return resp.json()  # list of comment dicts


# ──────────────────────────────────────────────────────────────────────────────
#  Public function
# ──────────────────────────────────────────────────────────────────────────────
def fetch_tiktok_search_with_comments(
    keyword: str,
    num_videos: int = 10,
    comments_per_video: int = 20,
) -> list[dict[str, Any]]:
    """
    Search TikTok and enrich each result with up to `comments_per_video` comments.

    Args:
        keyword (str): Search term.
        num_videos (int): How many videos to return (≤ 100, actor limit).
        comments_per_video (int): Comment count for each video (≤ 500).

    Returns:
        List[Dict]: Each dict is the original search item plus "comments".
    """
    results: list[dict[str, Any]] = []

    # 1️⃣ Search TikTok
    video_items = _search_tiktok(keyword, max_items=num_videos)

    # 2️⃣ For each video, grab comments
    for item in video_items[:num_videos]:
        # Different actors use slightly different field names; try common ones.
        video_url = (
            item.get("url")
            or item.get("videoUrl")
            or item.get("itemUrl")
            or item.get("shareUrl")
        )
        if not video_url:
            # Skip if we cannot find a usable URL.
            continue

        try:
            comments = _fetch_comments(video_url, limit=comments_per_video)
        except Exception as err:
            # Keep the video record even if comments fail, but mark the error.
            comments = []
            item["comment_error"] = str(err)

        # Attach comments to the original item
        item["comments"] = comments
        results.append(item)

    return results


# ──────────────────────────────────────────────────────────────────────────────
#  Example usage
# ──────────────────────────────────────────────────────────────────────────────

data = fetch_tiktok_search_with_comments(
    "ferrari testarossa", num_videos=10, comments_per_video=10
)
# Pretty-print first record
import json
import pprint

pprint.pprint(json.dumps(data[0], indent=2, ensure_ascii=False) if data else "No data")

# save the results to a file
with open("tiktok_search_results.json", "w", encoding="utf-8") as f:
    json.dump(data, f, indent=2, ensure_ascii=False)

('{\n'
 '  "url": "https://www.tiktok.com/@therealvarryx/video/7488448515195096342",\n'
 '  "id": "7488448515195096342",\n'
 '  "desc": "Gotta love the \\"fake F40\\" 😅 #ferrari #testarossa '
 '#koenigspecials ",\n'
 '  "createTime": "1743540293",\n'
 '  "scheduleTime": 0,\n'
 '  "video": {\n'
 '    "id": "7488448515195096342",\n'
 '    "height": 1024,\n'
 '    "width": 576,\n'
 '    "duration": 27,\n'
 '    "ratio": "540p",\n'
 '    "cover": '
 '"https://p16-common-sign-no.tiktokcdn-us.com/tos-no1a-p-0037-no/oo4xUzADPABzAEBQCE1sQyIw9iIBxeDAkSiECF~tplv-tiktokx-origin.image?dr=9636&x-expires=1749510000&x-signature=uMJCBvueX4OmFI1DbfImlLU9X7A%3D&t=4d5b0474&ps=13740610&shp=81f88b70&shcp=43f4a2f9&idc=useast8",\n'
 '    "originCover": '
 '"https://p16-pu-sign-no.tiktokcdn-eu.com/obj/tos-no1a-p-0037-no/oQBLfAP0GUjfQnNIXKAfnQQMsALI2hsDYZ8oET?lk3s=81f88b70&x-expires=1749510000&x-signature=8fnuY6wn%2BQ42cJklJM5RZwltkKI%3D&shp=81f88b70&shcp=-",\n'
 '    "dynamicCover": '
 '"https://p16-common-si

In [10]:
data[0]["comments"]

[{'videoWebUrl': 'https://www.tiktok.com/@therealvarryx/video/7488448515195096342',
  'submittedVideoUrl': 'https://www.tiktok.com/@therealvarryx/video/7488448515195096342',
  'input': 'https://www.tiktok.com/@therealvarryx/video/7488448515195096342',
  'cid': '7488954223842706198',
  'createTime': 1743658048,
  'createTimeISO': '2025-04-03T05:27:28.000Z',
  'text': 'the venturi 400gt is better.',
  'diggCount': 862,
  'likedByAuthor': False,
  'pinnedByAuthor': False,
  'repliesToId': None,
  'replyCommentTotal': 32,
  'uid': '7334804139882497056',
  'uniqueId': '2jz_pro',
  'avatarThumbnail': 'https://p16-pu-sign-no.tiktokcdn-eu.com/tos-no1a-avt-0068c001-no/7483ac5c52dcc9a957e09f5795ba18c4~tplv-tiktokx-cropcenter:100:100.jpg?dr=10399&refresh_token=274a8713&x-expires=1749423600&x-signature=21o2bSQEkou7aaXShi%2FXK7vbbGQ%3D&t=4d5b0474&ps=13740610&shp=30310797&shcp=ff37627b&idc=no1a'},
 {'videoWebUrl': 'https://www.tiktok.com/@therealvarryx/video/7488448515195096342',
  'submittedVideoUr